In [4]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading
import time
import csv

class Linecounter:
    def __init__(self, file_name):
        self.file_name = file_name
        self.line = []
        self.ip = []

    def read(self):
        with open(self.file_name, "r") as f:
            self.line = f.readlines()

    def fetch_ip(self):
        self.ip = [line.split(" ")[0] for line in self.line if line.strip()]
        return self.ip

    def ip_add_20(self):
        return [ip for ip in self.ip if ip.split(".")[0].isdigit() and int(ip.split(".")[0]) < 20]

    def ip_frequency(self):
        freq = {}
        for ip in self.ip:
            freq[ip] = freq.get(ip, 0) + 1
        return freq

class InteractiveLogAnalyzer:
    def __init__(self, root):
        self.root = root
        self.root.title("Interactive IP Log Analyzer")
        self.root.geometry("1000x600")
        self.file_path = None
        self.stop_thread = False

        self.setup_ui()

    def setup_ui(self):
        # File selection
        file_frame = tk.LabelFrame(self.root, text="Step 1: Select Log File")
        file_frame.pack(padx=10, pady=10, fill="x")

        self.file_label = tk.Label(file_frame, text="No file selected")
        self.file_label.pack(side="left", padx=10)

        tk.Button(file_frame, text="Browse File", command=self.load_file).pack(side="left", padx=10)

        # Filter options
        filter_frame = tk.LabelFrame(self.root, text="Step 2: Set Filters")
        filter_frame.pack(padx=10, pady=10, fill="x")

        tk.Label(filter_frame, text="Minimum Request Count:").grid(row=0, column=0, padx=5, pady=5, sticky="e")
        self.min_count_entry = tk.Entry(filter_frame, width=10)
        self.min_count_entry.insert(0, "1")
        self.min_count_entry.grid(row=0, column=1, padx=5, pady=5)

        self.private_var = tk.IntVar()
        tk.Checkbutton(filter_frame, text="Only Private IPs", variable=self.private_var).grid(row=0, column=2, padx=10)

        tk.Button(filter_frame, text="Start Analysis", command=self.start_analysis).grid(row=0, column=3, padx=10)

        # Treeview output
        self.tree = ttk.Treeview(self.root, columns=("IP", "Count"), show="headings", height=20)
        self.tree.heading("IP", text="IP Address")
        self.tree.heading("Count", text="Request Count")
        self.tree.column("IP", width=300)
        self.tree.column("Count", width=150)
        self.tree.pack(padx=10, pady=10, fill="both", expand=True)

        # Export button
        tk.Button(self.root, text="Export CSV", command=self.export_csv).pack(pady=10)

    def load_file(self):
        self.file_path = filedialog.askopenfilename(
            title="Select a log file",
            filetypes=[("Log/Text Files", "*.log *.txt"), ("All Files", "*.")]
        )
        if self.file_path:
            self.file_label.config(text=self.file_path)

    def start_analysis(self):
        if not self.file_path:
            messagebox.showerror("Error", "Please select a log file first.")
            return

        try:
            min_count = int(self.min_count_entry.get())
        except ValueError:
            messagebox.showerror("Error", "Minimum request count must be a number.")
            return

        self.analyze_log(min_count, self.private_var.get())

    def is_private_ip(self, ip):
        return ip.startswith("10.") or ip.startswith("172.") or ip.startswith("192.168.") or ip.startswith("127.")

    def analyze_log(self, min_count, only_private):
        analyzer = Linecounter(self.file_path)
        analyzer.read()
        analyzer.fetch_ip()
        freq = analyzer.ip_frequency()

        self.tree.delete(*self.tree.get_children())

        for ip, count in sorted(freq.items(), key=lambda x: x[1], reverse=True):
            if count < min_count:
                continue
            if only_private and not self.is_private_ip(ip):
                continue
            self.tree.insert("", "end", values=(ip, count))

    def export_csv(self):
        if not self.file_path:
            messagebox.showerror("No File", "Please load a log file first.")
            return

        file = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV File", "*.csv")])
        if not file:
            return

        try:
            with open(file, "w", newline="") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(["IP Address", "Count"])
                for row_id in self.tree.get_children():
                    writer.writerow(self.tree.item(row_id)['values'])

            messagebox.showinfo("Exported", f"Data exported successfully to {file}")
        except Exception as e:
            messagebox.showerror("Error", str(e))

if __name__ == "__main__":
    root = tk.Tk()
    app = InteractiveLogAnalyzer(root)
    root.mainloop()